In [ ]:
import sys
sys.path.insert(0, '/Users/gil-arnaudcoche/Documents/ijoutaku/')

import time
import keras
import matplotlib.pyplot as plt


from config.paths import TRAIN_PATH
from config.paths import MODELS_PATH

from config.models import EPOCHS
from config.models import CONV_FILTERS
from config.models import CONV_KERNEL
from config.models import LSTM_FILTERS
from config.models import LSTM_KERNEL
from config.models import PADDING
from config.models import SIGMOID
from config.models import RELU
from config.models import STRIDES
from config.models import LOSS
from config.models import LEARNING_RATE
from config.models import EPSILON

from lib.generators import SequenceDataGenerator

In [ ]:
train_set = SequenceDataGenerator(TRAIN_PATH)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(CONV_FILTERS, CONV_KERNEL, activation=RELU, strides=STRIDES, padding=PADDING), batch_input_shape=(SequenceDataGenerator._BATCH_SIZE, int(SequenceDataGenerator._SEQUENCE_SIZE/SequenceDataGenerator._STRIDE_KERNEL), SequenceDataGenerator._IMAGE_HEIGHT, SequenceDataGenerator._IMAGE_WIDTH, 1)))
model.add(keras.layers.LayerNormalization())
# # # # #
model.add(keras.layers.ConvLSTM2D(LSTM_FILTERS, LSTM_KERNEL, padding=PADDING, return_sequences=True))
model.add(keras.layers.LayerNormalization())
# # # # #
model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(CONV_FILTERS, CONV_KERNEL, activation=RELU, strides=STRIDES, padding=PADDING)))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(1, CONV_KERNEL, activation=SIGMOID, padding=PADDING)))
print(model.summary())
model.compile(loss=LOSS, optimizer=keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON))
start = time.time()
history = model.fit(train_set, batch_size=SequenceDataGenerator._BATCH_SIZE, epochs=EPOCHS, shuffle=False)
end = time.time()
duration_seconds = int(end - start)
duration_minutes, leftover_seconds = duration_seconds // 60, duration_seconds % 60
print(f'Training took {duration_minutes} min {leftover_seconds} s')
model.save(f'{MODELS_PATH}lstm-convolutional-autoencoder.hdf5')


In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()